In [0]:
stop

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'our',
 'ourselv',
 'you',
 "you'r",
 "you'v",
 "you'll",
 "you'd",
 'your',
 'your',
 'yourself',
 'yourselv',
 'he',
 'him',
 'hi',
 'himself',
 'she',
 "she'",
 'her',
 'her',
 'herself',
 'it',
 "it'",
 'it',
 'itself',
 'they',
 'them',
 'their',
 'their',
 'themselv',
 'what',
 'which',
 'who',
 'whom',
 'thi',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'wa',
 'were',
 'be',
 'been',
 'be',
 'have',
 'ha',
 'had',
 'have',
 'do',
 'doe',
 'did',
 'do',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'becaus',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'dure',
 'befor',
 'after',
 'abov',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'onc',
 'here',
 'there',
 'when',
 'where',
 'whi',
 'how',
 'all',
 'ani',
 'both',
 'each',
 'few',
 'more',
 'most',
 'oth

# Predict sentiment score of moview review with Logistic Regression

## 1.Import data & Basic library

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
sns.set()

In [0]:
df = pd.read_csv('/content/drive/My Drive/FTMLE - Tonga/Data/movie_review.csv',encoding='utf-8', sep='\t')

In [0]:
# Check sample of the dataset
df.sample(5)

,id,review,sentiment
11403,9886_4,This is Peter Falk's film. Period.<br /><br />...,0
10858,10929_1,This movie was the beatliest mormon movie made...,0
18248,9648_8,I liked this film very much. The story jumps b...,1
4413,3593_8,"Michelle Rodriguez plays Diana, a high school ...",1
13821,11905_8,William Castle is notorious among horror fans ...,1


## 2.Overview of the data





In [0]:
# Check content of 1 comment for the meaning of sentiment
# 0 means negative ; 1 means positive
df.loc[9715,'review']


'Three giant sabretooth tigers(..created in a laboratory from mitochondrial DNA, a \\genetic breakthrough\\" derived from fossil material)are on the rampage accidentally set free through a series of events(such as a computer geek\'s introduced virus in order to unlock security measures keeping the resort novelty shops closed during construction & a security guard\'s leaving a gate open while searching for the missing page from a porn mag that flew away in the breeze)that threaten the lives of those it comes in contact with. The tigers are always hungry, but are unable to digest what they eat. So pretty much the tigers just rip their prey to shreds. Victims include a group of college kids(..the stereotypes include a goth girl, jock and tech nerd), security personnel, and those somewhat developed rich scoundrels who we can easily despise and wish horrible death.<br /><br />Rounding out a series of bad sci-fi channel flicks, Attack of the Sabretooth has some of the most wretched computer 

In [0]:
df.info()
# There is no missing data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22500 entries, 0 to 22499
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         22500 non-null  object
 1   review     22500 non-null  object
 2   sentiment  22500 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 527.5+ KB


In [0]:
# read 1  review to check the meaning of sentiment 
list(df.loc[df['id'] == '8369_8','review'])

# The review also contains some special character like '<br />', 

['Hrm-I think that line was from the old movie posters.<br /><br />This is a dumb movie that seems to have been translated from some language that was totally unfamiliar to the translator. Here\'s a tip: Any movie that starts with a black screen and text reading \\In the future...\\" is going to be fun. This means that the premise is so implausible that they have to explain it to you.<br /><br />In this case, \\"In the future...\\" means that, instead of fighting wars, nations have guys climb into giant robots and duke it out to determine, well, that\'s never terribly clear, but it\'s probably something really important. There are good guys (obviously capitalists, i.e. \\"us\\") and bad guys (Commies!) and there are big stop-motion robots.<br /><br />Sadly, the effects budget was pretty slim, so we don\'t get to see a lot of the big robots. There are plenty of cheap looking interior scenes, and then a big space fight near the end. The space fight is especially nice, as it serves precis

In [0]:
df['sentiment'].value_counts()

# There is only 2 tye of sentiment, 0 and 1. 
# 0 means negative ; 1 means positive

1    11278
0    11222
Name: sentiment, dtype: int64

In [0]:
# Check for duplicate id ->  No duplicate found
df['id'].count() == df['id'].nunique()

# It does not support our prediction model so we can remove it 
df.drop(labels='id',axis=1,inplace = True)

In [0]:
df

,review,sentiment
0,With all this stuff going down at the moment w...,1
1,"\The Classic War of the Worlds\"" by Timothy Hi...",1
2,The film starts with a manager (Nicholas Bell)...,0
3,It must be assumed that those who praised this...,0
4,Superbly trashy and wondrously unpretentious 8...,1
...,...,...
22495,It seems like more consideration has gone into...,0
22496,I don't believe they made this film. Completel...,0
22497,"Guy is a loser. Can't get girls, needs to buil...",0
22498,This 30 minute documentary BuÃ±uel made in the...,0


In [0]:
# Check for duplicate review -> there are duplicated review (could be fake reviews)
# We will check some sample of duplicate review
df['review'].count() == df['review'].nunique()


False

In [0]:
# Sort the df base Columns review & Separate the duplicated review to check
df.sort_values('review', inplace = True)
df

,review,sentiment
7381,A Turkish Bath sequence in a film noir loc...,1
318,!!!! POSSIBLE MILD SPOILER !!!!!<br /><br />As...,0
5741,!!!!! OF COURSE THERE'S SPOILERS !!!!! I'm sur...,0
7836,!!!!! POSSIBLE SPOILER !!!!!<br /><br />You`d ...,0
378,### Spoilers! ### <br /><br />What is this mov...,0
...,...,...
3403,you must be seeing my comments over many films...,0
19068,zero day is based of columbine high school mas...,1
17180,{Possible spoilers coming up... you've been fo...,1
10232,{rant start} I didn't want to believe them at ...,0


In [0]:
check = df[df.duplicated()]

In [0]:
check
# Look like therer is no duplicate review

,review,sentiment
13252,'Dead Letter Office' is a low-budget film abou...,0
6311,".......Playing Kaddiddlehopper, Col San Fernan...",1
19066,"<br /><br />Back in his youth, the old man had...",0
6658,A have a female friend who is currently being ...,1
4441,"A longtime fan of Bette Midler, I must say her...",1
...,...,...
2783,"Wow, here it finally is; the action \movie\"" w...",0
20989,You do realize that you've been watching the E...,0
2415,"in this movie, joe pesci slams dunks a basketb...",0
20020,it's amazing that so many people that i know h...,1


# Prediction model

## Step 1: Data Cleaning

In [0]:
# The special character could be recognize as part of the word & confuse our model
# So we need to clean these special characters (astrophoe,the comma,..)
df.sample(5)

,review,sentiment
19023,I thought Hedy Burress (who managed to escape ...,0
10478,This is even worse than the original Game of D...,0
16381,Even by 1942 standards of movie-making the set...,0
21254,"After viewing the film, I was truly shocked to...",0
10590,Early film directed by D.W. Griffith; it featu...,0


In [0]:
# Overall look at the most common words
from collections import Counter
vocab = Counter()

for document in df['review']:
  for word in document.split(' '):
    vocab[word] += 1

In [0]:
vocab.most_common(20)

[('the', 258519),
 ('a', 139707),
 ('and', 137397),
 ('of', 128750),
 ('to', 119278),
 ('is', 92935),
 ('in', 77245),
 ('I', 59255),
 ('that', 57991),
 ('this', 51379),
 ('it', 48865),
 ('/><br', 45851),
 ('was', 42004),
 ('as', 38288),
 ('with', 37496),
 ('for', 36919),
 ('The', 30399),
 ('but', 30350),
 ('on', 27738),
 ('movie', 27342)]

In [0]:
# Initiate stop word on current vocab so check for error
from nltk.corpus import stopwords

stop = stopwords.words('english')
vocab_reduced = Counter()

for word, count in vocab.items():
  if not word in stop:
    vocab_reduced[word] = count

vocab_reduced.most_common(10)

[('I', 59255),
 ('/><br', 45851),
 ('The', 30399),
 ('movie', 27342),
 ('film', 24768),
 ('one', 18704),
 ('like', 16278),
 ('This', 11074),
 ('would', 10720),
 ('good', 10243)]

In [0]:
# download stop word 
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
# Create tokennizer function
from nltk.stem import PorterStemmer

porter = PorterStemmer()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

def tokenizer_stop(text):
    return [porter.stem(word) for word in text]

## Step 2 : Representation

In [0]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer


#Tokenize the stop word
stop = tokenizer_stop(stop)

def preprocessor(text):
    """ Return a cleaned version of text
    """
    # Remove HTML markup
    text = re.sub('<[^>]*>', '', text)
    # Save emoticons for later appending
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ''))
    
    return text

tfidf = TfidfVectorizer(stop_words=stop,
                        tokenizer=tokenizer_porter,
                        preprocessor=preprocessor)

In [0]:
# Test our processing funtion on 1 random data
print(df.loc[8333,'review'])

preprocessor(df.loc[8333,'review'])

To \Bend It Like Beckham\" may not mean much to us Americans who know very little about the other football (soccer), but to English sports fans, it is equivalent to \"Hit it Like Bonds\" or \"Dunk it Like Jordan.\" Any young soccer player dreams of bending a soccer ball around one player and into the net for a goal, much like star player David Beckham does, much like the young Indian girl, Jess (Parminder Nagra), does in the film Bend It Like Beckham. Jess loves to play pick up soccer games, the kind forbidden by her traditionalist mother. However, while playing one day, a passing friend named Jules (Keira Knightley) sees her play and invites her to try out for a traveling, all girls soccer team. After satisfying the coach Joe (Jonathan Rhys-Meyers), she makes the team, something she knows her mother would not approve of. The movie is not about disobeying parents, but rather a girl doing what she wants to do, even if that goes against culture, not just the parents. There is humor throw

'to bend it like beckham may not mean much to us americans who know very little about the other football soccer but to english sports fans it is equivalent to hit it like bonds or dunk it like jordan any young soccer player dreams of bending a soccer ball around one player and into the net for a goal much like star player david beckham does much like the young indian girl jess parminder nagra does in the film bend it like beckham jess loves to play pick up soccer games the kind forbidden by her traditionalist mother however while playing one day a passing friend named jules keira knightley sees her play and invites her to try out for a traveling all girls soccer team after satisfying the coach joe jonathan rhys meyers she makes the team something she knows her mother would not approve of the movie is not about disobeying parents but rather a girl doing what she wants to do even if that goes against culture not just the parents there is humor thrown throughout the movie especially when 

## Step 3: Classification

In [0]:
# split the dataset in train and test

from sklearn.model_selection import train_test_split

X = df['review']

y = df['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y , test_size = 0.2)

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

# A pipeline is what chains several steps together, once the initial exploration is done. 
# For example, some codes are meant to transform features — normalise numericals, or turn text into vectors, 
# or fill up missing data, they are transformers; other codes are meant to predict variables by fitting an algorithm,
# they are estimators. Pipeline chains all these together which can then be applied to training data
clf = Pipeline([('vect', tfidf),
                ('clf', LogisticRegression(random_state=0))])

clf.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['becau', 'r', 'v'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=<function preprocessor at 0x7fe12e8b2378>,
                                 smooth_idf=True,
                                 stop_words=['i', 'me', 'my', 'myself', '...
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenizer_porter at 0x7fe12efdeae8>,
                                 use_idf=True, vocabulary=None)),
                ('clf',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
         

In [0]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Now apply those above metrics to evaluate your model

predictions = clf.predict(X_test)
accuracy_score(y_test, predictions)

0.8848888888888888

In [0]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.90      0.87      0.88      2267
           1       0.87      0.90      0.89      2233

    accuracy                           0.88      4500
   macro avg       0.89      0.89      0.88      4500
weighted avg       0.89      0.88      0.88      4500



In [0]:
# Create dataframe of test data & prediction result
pred_result = pd.DataFrame(data={'test_data': list(y_test) , 'predict_result': list(predictions)})

In [0]:
# Export the pred_result to csv
pred_result.to_csv('tuananh.csv')

## Model testing

In [0]:
review_test = [
    "Not the worst spoof movie that's been made, but it is a big disappointment.",
    "I love this but would not recommend the movie for children",
    ":) this movie sucks so badly",
]

preds = clf.predict_proba(review_test)

for i in range(len(review_test)):
    print(f'{i} --> Negative, Positive = {preds[i]}')

0 --> Negative, Positive = [0.99407121 0.00592879]
1 --> Negative, Positive = [0.16647525 0.83352475]
2 --> Negative, Positive = [0.84328114 0.15671886]
